# Alzeihmer desease detection using volume patches 

In [1]:
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
from pathlib import Path

path = "kaggle/input/adni-data"
path = path + "\ADNI_PROCESSED"

def apply_mask(img_n_mmni, img_mask):
    """
        Taking a n_mmni and apply the correspondant mask
        param:
            img_n_mmi   : image n_mmi
            img_mask    : mask
    """
    mmni_m = img_n_mmni.get_fdata()
    mask_m = img_mask.get_fdata().astype(bool)
    mask_bg = np.logical_not(mask_m)
    mmni_m[mask_bg] = 0
    return mmni_m

def process_irm_data():
    """
        Create a new directory and process all images from tha ADNI1 directory
    """
    path = str(Path().resolve())
    path_res = path + "\\ADNI_PROCESSED"
    Path(path_res).mkdir(parents=True, exist_ok=True) # Create a directory for data processed
    path = path + "\\ADNI1"
    for filename in os.listdir(path):
        if filename.startswith("n_mmni"):
            n_mmni_filename = os.path.join(path, filename)
            mask_filename = os.path.join(path, "mask_" + filename)
            img_n_mmni = nib.load(n_mmni_filename)
            img_mask = nib.load(mask_filename)
            n_mmni_mask = apply_mask(img_n_mmni, img_mask)
            img = nib.Nifti1Image(n_mmni_mask, np.eye(4))
            nib.save(img, os.path.join(path_res, filename))

In [2]:
def cut_2D_i(img_n_mmni, axe, idx):
    """
        Function that returns a 2D cut from the "img" in the index "idx", along the axe given in parameter
    """
    axe_dim = {"x": img_n_mmni.shape[0], "y": img_n_mmni.shape[1], "z":img_n_mmni.shape[2]}
    if axe_dim[axe] <= idx or idx < 0:
        print("Invalid value for index must be between 0 and " , axe_dim[axe])
        return
    if axe == "x":
        cropped_img = img_n_mmni.slicer[idx:idx+1, 18:199, :]
        img_data = cropped_img.get_fdata()
        img_data = np.transpose(img_data, (2, 1, 0)) / 255.0
        img_data = np.transpose(img_data, (1, 0, 2)) / 255.0
    elif axe == "y":
        cropped_img = img_n_mmni.slicer[45:145, idx:idx+1,35:135] #eliminate black borders
        img_data = cropped_img.get_fdata()
        img_data = np.transpose(img_data, (0, 2, 1)) / 255.0
    elif axe == "z":
        cropped_img = img_n_mmni.slicer[..., idx:idx+1]
        img_data = cropped_img.get_fdata()
    else:
        print("Choose a valid value for axe: x, y or z")

    return img_data

def custom_patch_3D(img_n_mmni, x_tup, y_tup, z_tup):
    axe_dim = {"x": img_n_mmni.shape[0], "y": img_n_mmni.shape[1], "z":img_n_mmni.shape[2]}
    if axe_dim["x"] <= x_tup[1] or x_tup[0] < 0 or axe_dim["y"] <= y_tup[1] or y_tup[0] < 0 or axe_dim["z"] <= z_tup[1] or z_tup[0] < 0 :
        print("Invalid values")
        return 
    else:
        cropped_img = img_n_mmni.slicer[x_tup[0]:x_tup[1], y_tup[0]:y_tup[1], z_tup[0]:z_tup[1]]
        img_data = cropped_img.get_fdata()
        img_data.resize(img_data.shape + (1, ))
        return img_data

In [3]:
import os
import re
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

def load_data(path):
    data = pd.read_csv(path, names= ['Subject ID', 'Rooster ID', 'Age', 'Sexe', 'Group', 'Conversion', 'MMSE', 'RAVLT', 'FAQ', 'CDR-SB', 'ADAS11'], usecols = ['Subject ID', 'Rooster ID', 'Group'])
    data.index = data['Subject ID']
    data = data.drop(['Subject ID'], axis=1)
    data = data[(data.Group == 'CN') | (data.Group == 'AD')]
    return data

path = "../input/adni-data"
path = path + "/list_standardized_tongtong_2017.csv"
y_data = load_data(path)
y_data.head(7)

In [4]:
from tensorflow.keras.utils import to_categorical
# x y z hypocamp = 
#x = 40, 80   autre  140
#y = 90, 130  autre 132
#z = 40, 80   autre 77
usecols = ['Subject ID', 'Rooster ID', 'Group']
# ['CN', 'AD']
def prepare_X_of_Y_3D(Y):
    X_data = []
    Y_data = []
    X_test_index = []
    Y_test = []
    path = "../input/adni-data/ADNI_PROCESSED"
    n_test_AD = 0
    n_test_CN = 0
    #f = open(path, "r")
    for index, row in Y.iterrows():
        file = path + '/n_mmni_fADNI_' + index + '_1.5T_t1w.nii'
        if os.path.isfile(file):
            img_n_mmni = nib.load(file)
            # Taking 4 images for test purpose
            if (Y['Group'][index] == 'AD' and n_test_AD < 2) or (Y['Group'][index] == 'CN' and n_test_CN < 2):
                n_test_AD += 1 if Y['Group'][index] == 'AD' else n_test_AD
                n_test_CN += 1 if Y['Group'][index] == 'CN' else n_test_CN
                X_test_index.append(index)
            else:
                img_data_left_right = []
                
                #left hyppocampus
                img_data = custom_patch_3D(img_n_mmni, x_tup=(40, 80), y_tup=(90, 130), z_tup=(40, 80))
                img_data_left_right.append(img_data)
                
                #right hyppocampus
                img_data = custom_patch_3D(img_n_mmni, x_tup=(100,140), y_tup=(90, 130), z_tup=(40, 80))
                img_data_left_right.append(img_data)
        
                #append left and right hyppocamus to data
                X_data.append(img_data_left_right)
            
                if Y['Group'][index] == 'AD':
                    Y_data.append(1)
                elif Y['Group'][index] == 'CN':
                    Y_data.append(0)
        else:
            Y.drop(index, inplace=True)
    return np.array(X_data), Y_data, X_test_index
'''
if os.path.isfile('X_data_3D.npy') and os.path.isfile('Y_data_3D.npy') and os.path.isfile('X_test_index_3D.npy'):
    X_data_3D = np.load('X_data_3D.npy')
    Y_data_3D = np.load('Y_data_3D.npy')
    X_test_index_3D = np.load('X_test_index_3D.npy')
else:
'''
X_data_3D, Y_data_list_3D, X_test_index_3D = prepare_X_of_Y_3D(y_data)
Y_data_3D = to_categorical(Y_data_list_3D, num_classes=2)
    
print(len(X_data_3D) == len(Y_data_3D))
print(len(X_data_3D))

### Data augmentation by adding right hyppocampus (mirrored) to data with the left one.

In [ ]:
from tensorflow.keras.utils import to_categorical
# x y z hypocamp = 
#x = 40, 80   autre  140
#y = 90, 130  autre 132
#z = 40, 80   autre 77
usecols = ['Subject ID', 'Rooster ID', 'Group']
# ['CN', 'AD']
def prepare_X_of_Y_3D_left(Y):
    X_data = []
    Y_data = []
    X_test_index = []
    Y_test = []
    path = "../input/adni-data/ADNI_PROCESSED"
    n_test_AD = 0
    n_test_CN = 0
    #f = open(path, "r")
    for index, row in Y.iterrows():
        file = path + '/n_mmni_fADNI_' + index + '_1.5T_t1w.nii'
        if os.path.isfile(file):
            img_n_mmni = nib.load(file)
            # Taking 4 images for test purpose
            if (Y['Group'][index] == 'AD' and n_test_AD < 2) or (Y['Group'][index] == 'CN' and n_test_CN < 2):
                n_test_AD += 1 if Y['Group'][index] == 'AD' else n_test_AD
                n_test_CN += 1 if Y['Group'][index] == 'CN' else n_test_CN
                X_test_index.append(index)
            else:
                img_data = custom_patch_3D(img_n_mmni, x_tup=(90,130), y_tup=(90, 130), z_tup=(50, 110))
                X_data.append(img_data)
                if Y['Group'][index] == 'AD':
                    Y_data.append(1)
                elif Y['Group'][index] == 'CN':
                    Y_data.append(0)
        else:
            Y.drop(index, inplace=True)
    return np.array(X_data), Y_data, X_test_index

def prepare_X_of_Y_3D_right(Y):
    X_data = []
    Y_data = []
    X_test_index = []
    Y_test = []
    path = "../input/adni-data/ADNI_PROCESSED"
    n_test_AD = 0
    n_test_CN = 0
    #f = open(path, "r")
    for index, row in Y.iterrows():
        file = path + '/n_mmni_fADNI_' + index + '_1.5T_t1w.nii'
        if os.path.isfile(file):
            img_n_mmni = nib.load(file)
            # Taking 4 images for test purpose
            if (Y['Group'][index] == 'AD' and n_test_AD < 2) or (Y['Group'][index] == 'CN' and n_test_CN < 2):
                n_test_AD += 1 if Y['Group'][index] == 'AD' else n_test_AD
                n_test_CN += 1 if Y['Group'][index] == 'CN' else n_test_CN
                X_test_index.append(index)
            else:
                img_data = custom_patch_3D(img_n_mmni, x_tup=(40, 80), y_tup=(90, 130), z_tup=(50, 110))
                img_data = img_data.reshape((40 ,40, 60, 1))
                img_data = np.transpose(img_data, (1, 0, 2, 3))
                img_data = img_data.reshape((40 ,40, 60, 1))
                
                X_data.append(img_data)
                if Y['Group'][index] == 'AD':
                    Y_data.append(1)
                elif Y['Group'][index] == 'CN':
                    Y_data.append(0)
        else:
            Y.drop(index, inplace=True)
    return np.array(X_data), Y_data, X_test_index

X_data_3D_right, Y_data_list_3D_right, X_test_index_3D_right = prepare_X_of_Y_3D_right(y_data)
X_data_3D_left, Y_data_list_3D_left, X_test_index_3D_left = prepare_X_of_Y_3D_right(y_data)

X_data_3D = np.concatenate((X_data_3D_right, X_data_3D_left))
X_test_index_3D =Y_data_list_3D = np.concatenate((X_test_index_3D_right, X_test_index_3D_left))

Y_data_3D_left = to_categorical(Y_data_list_3D_left, num_classes=2)
Y_data_3D_right = to_categorical(Y_data_list_3D_right, num_classes=2)

Y_data_3D = np.concatenate((Y_data_3D_right, Y_data_3D_left))

print(len(X_data_3D) == len(Y_data_3D))
print(len(X_data_3D))

### Left Hyppocampus example

In [9]:
from nibabel.viewers import OrthoSlicer3D
OrthoSlicer3D(X_data_3D[1][0]).show()

### Right Hyppocampus example

In [10]:
from nibabel.viewers import OrthoSlicer3D
OrthoSlicer3D(X_data_3D[1][1]).show()

## Loss visualization function
This function is used to visualize the variations of loss, val_loss, accuracy and val_accuracy over epochs. It is used for 3D and 2D models. 

In [5]:
import matplotlib.pyplot as plt

# Plot the validation and training data separately
def plot_loss_curves(history):
      """
      Returns separate loss curves for training and validation metrics.
      """ 
      loss = history.history['loss']
      val_loss = history.history['val_loss']

      accuracy = history.history['accuracy']
      val_accuracy = history.history['val_accuracy']

      epochs = range(len(history.history['loss']))

      # Plot loss
      plt.plot(epochs, loss, label='training_loss')
      plt.plot(epochs, val_loss, label='val_loss')
      plt.title('Loss')
      plt.xlabel('Epochs')
      plt.legend()

      # Plot accuracy
      plt.figure()  
      plt.plot(epochs, accuracy, label='training_accuracy')
      plt.plot(epochs, val_accuracy, label='val_accuracy')
      plt.title('Accuracy')
      plt.xlabel('Epochs')
      plt.legend()
      plt.savefig('accuracy.png')

### 3D implementation of classification model
- Creation of an auto-encoder model and an only encoder model.
- compilation of the model using adam optimizer, and binary_crossentropy loss.- fiting the model using GPU if it exists.

In [6]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, UpSampling2D, Cropping2D, Conv3D, MaxPooling3D, UpSampling3D, Cropping3D, Input, concatenate, Flatten, Dense, Dropout, BatchNormalization, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalAveragePooling3D, add
from tensorflow.keras.models import Model

def create_Unet_model3D_encoder(input_size, depth=5, padding='valid', nb_class=2):
    inputs = Input(shape=input_size)
    x = inputs
    num_filters = 64
    for i in range(depth):
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        if i != depth - 1:
            x = MaxPooling3D(pool_size=(2,2,2), strides=2)(x)
            num_filters *= 2
    
        x = Dropout(0.6)(x)
    
    x = Flatten()(x)
    # x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', kernel_regularizer='l2')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    
    x = Dense(128, activation='relu', kernel_regularizer='l2')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    
    outputs = Dense(nb_class, activation='softmax')(x)

    return Model(inputs, outputs)

In [9]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, UpSampling2D, Cropping2D, Conv3D, MaxPooling3D, UpSampling3D, Cropping3D, Input, Concatenate, Flatten, Dense, Dropout, BatchNormalization, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalAveragePooling3D, add

def intermediate_network(inputs, i, depth, padding='same'):
    x = inputs[:,i,:,:,:,:]
    
    num_filters = 32
    for i in range(depth):
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv3D(filters=num_filters, kernel_size=(3,3,3), padding=padding)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        if i != depth - 1:
            x = MaxPooling3D(pool_size=(2,2,2), strides=2)(x)
            num_filters *= 2
    
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    return x

def create_model_3D(input_size, depth=2, nb_class=2):
    inputs = Input(shape=input_size)
    nb_slices = 2
    out_list = []
    for i in range(nb_slices):
        out_list.append(intermediate_network(inputs=inputs, i=i, depth=depth, padding='same'))

    x = Concatenate()(out_list)
    x = Flatten()(x)
    
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x)
    
    
    x = Dense(128, activation='relu', kernel_regularizer='l2')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    outputs = Dense(nb_class, activation='softmax')(x)
    
    return Model(inputs, outputs)

In [13]:
import tensorflow

OPT    = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)#0.0001

model_3 = create_model_3D(X_data_3D[0].shape, depth=2, nb_class=2)
model_3.compile(optimizer=OPT, loss='binary_crossentropy', metrics=['accuracy'])
model_3.summary()



In [ ]:
import tensorflow

OPT    = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)#0.0001

model_3 = create_Unet_model3D_encoder(X_data_3D[0].shape, depth=2)
model_3.compile(optimizer=OPT, loss='binary_crossentropy', metrics=['accuracy'])
model_3.summary()


In [ ]:
X_data_3D[1:200].shape

In [12]:
from sklearn.model_selection import train_test_split

# reduce amount of data if GPU can't handle: X_data_3D[0:100], Y_data_3D[0:100]
X_train_3D, X_val_3D, Y_train_3D, Y_val_3D = train_test_split(X_data_3D, Y_data_3D, test_size=0.2,random_state=42)
print("Data splited")

In [14]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=20)
history = model_3.fit(np.asarray(X_train_3D), np.asarray(Y_train_3D), epochs= 15, batch_size=16, validation_data=(X_val_3D, Y_val_3D), callbacks=[es])

In [ ]:
plot_loss_curves(history)